# Unitary ansatz entering the VQE

- Check performance of UCCSD / QCC compared to FCI
- Pick orbitals for larger basis set (6-31G)
- Select engangular set size for QCC by effective gradient

In [2]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients
trotter_steps = 1

### LiH in STO-3G basis

- FCI

In [2]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci', backend='qulacs_gpu')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 12 

FCI energy: -7.882362286810955



- UCCSD
 - Used GpyOpt because Scipy took too long

In [3]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.optimizers.optimizer_gpyopt.minimize(objective=E, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, backend='qulacs', maxiter=300)
print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 24 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_gpyopt.OptimizerGPyOpt'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

method          : lbfgs
Objective       : 1 expectationvalues
num acquisition: 1, time elapsed: 0.46s
num acquisition: 2, time elapsed: 1.36s
num acquisition: 3, time elapsed: 2.12s
num acquisition: 4, time elapsed: 3.19s
num acquisition: 5, time elapsed: 4.02s
num acquisition: 6, time elapsed: 4.78s
num acquisition: 7, time elapsed: 5.44s
num acquisition: 8, time elapsed: 6.30s
num acquisition: 9, time elapsed: 7.18s
num acquisition: 10, time elapsed: 7.98s
num acquisition: 11, time elapsed: 8.52s
num acquisition: 12, time elapsed: 9.05s
num acquisition: 13, time elapsed: 9.75s
num acquisition: 14, time elapsed: 10.28s
num acquisition: 15, time elapsed: 10.93s
num acquisition: 16, time elapsed: 11.54s
num acquisition: 17, time elapse

- QCC

In [4]:
hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()



#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*lih.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.123
2 : 0.0335
3 : 0.0335
4 : 0.024
5 : 0.024
6 : 0.0215
7 : 0.0121
8 : 0.0098
9 : 0.0098
10 : 0.0076
11 : 0.0076
12 : 0.0076
13 : 0.0076
14 : 0.0072
15 : 0.002
16 : 0.002
17 : 0.0013
18 : 0.0013
19 : 0.0012
20 : 0.0012
21 : 0.0007
22 : 0.0007
23 : 0.0004
24 : 0.0004
25 : 0.0002
26 : 0.0002
27 : 0.0
28 : 0.0
29 : 0.0
30 : 0.0
31 : 0.0
32 : 0.0


#### We see that selecting 14 orbitals will be required, but eventually it does not work very well

In [5]:
n_ents = 14
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)


U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))


Selected entanglers:
1.0 [X2 Y3 X10 X11]
1.0 [X2 Y3 X5 X10]
1.0 [X2 Y3 X4 X11]
1.0 [X2 Y3 X6 X7]
1.0 [X2 Y3 X8 X9]
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X6 X7]
1.0 [X0 Y1 X8 X9]
1.0 [X0 Y3 X6 X7]
1.0 [X0 Y3 X8 X9]
1.0 [X1 Y2 X6 X7]
1.0 [X1 Y2 X8 X9]
1.0 [X0 Y1 X10 X11]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 76 expectationvalues

active variables : 38

E=-6.25839976  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11

#### We instead working on 6-31G basis and select orbitals to be used in the calculation

In [5]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='6-31g'
active = {'A1': [0, 1], 'B1': [0], 'B2': [0]}
lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

E_FCI = lih.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -7.977129915419726


- UCCSD
 - Calculation cost is cheaper than in STO-3 full orbital case

In [6]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6, backend='qulacs', method='bfgs')
print('\nObtained UCCSD energy: {}'.format(result.energy))



Hamiltonian has 105 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 256 expectationvalues

active variables : 8

E=-7.97685913  angles= {(3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (2, 0, 2, 1): 0.0, (3, 0, 3, 1): 0.0, (2, 1, 2, 0): 0.0, (3, 1, 3, 0): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-7.97706708  angles= {(3, 0, 3, 0): -0.0006241798400878906, (2, 1, 2, 1): -0.01934528350830078, (3, 1, 3, 1): -0.01934528350830078, (2, 0, 2, 1): 0.0015673637390136719, (3, 0, 3, 1): 0.0015673637390136719, (2, 1, 2, 0): 0.0015673637390136719, (3, 1, 3, 0): 0.0015673637390136719, (2, 0, 2, 0): -0.0006241798400878906}  samples= None
E=-7.97682716  angles= {(3, 0, 3, 0): 0.0039079496940328425, (2, 1, 2, 1): -0.017093156109609754,

- QCC
 - Has consistency with FCI
 - ~100x performance boost compared to UCCSD

In [7]:
hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


#Define number of entanglers to enter ansatz

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*lih.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))



Hamiltonian has 105 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0097
2 : 0.0097
3 : 0.0008
4 : 0.0008
5 : 0.0008
6 : 0.0008
7 : 0.0003
8 : 0.0003


#### We only need 2 engangulars this case.

In [9]:
n_ents = 2
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

#Mean-field part of U (Omega):
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))



Selected entanglers:
1.0 [X2 Y3 X4 X5]
1.0 [X2 Y3 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 36 expectationvalues

active variables : 18

E=-7.97685913  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0}  samples= None
E=-7.97701384  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592415171214, gamma_2: 0.0, beta_3: 3.141592415171214, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.00967264175

### H4 in 6-31G basis

- FCI

In [3]:
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)

basis = '6-31g'
active = { 'AG':[0,1], 'B1U': [0], 'B2U': [0]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals=active)

print(list(map(lambda item: item[1][0], h4.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, h4.orbitals_by_irrep.items())))

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
[QuantumChemistryPsi4.OrbitalData(irrep='AG', idx_irrep=0, idx_total=0, energy=-0.6577756529605916), QuantumChemistryPsi4.OrbitalData(irrep='B1U', idx_irrep=0, idx_total=1, energy=-0.3241801225581642), QuantumChemistryPsi4.OrbitalData(irrep='B2U', idx_irrep=0, idx_total=2, energy=-0.021997174708868056), QuantumChemistryPsi4.OrbitalData(irrep='B3G', idx_irrep=0, idx_total=3, energy=0.2489349600035358)]
[-0.6577756529605916, -0.3241801225581642, -0.021997174708868056, 0.2489349600035358]
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -1.9783982153570014


- UCCSD

In [4]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 105 terms


Number of UCCSD amplitudes: 4 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 128 expectationvalues

active variables : 4

E=-1.93303733  angles= {(3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-1.92926868  angles= {(3, 0, 3, 0): -0.0867493599653244, (2, 1, 2, 1): -0.09617340564727783, (3, 1, 3, 1): -0.06707680225372314, (2, 0, 2, 0): -0.20908880233764648}  samples= None
E=-1.94840960  angles= {(3, 0, 3, 0): -0.04088724847850404, (2, 1, 2, 1): -0.04532904836757326, (3, 1, 3, 1): -0.031615056087880414, (2, 0, 2, 0): -0.09854903619657156}  samples= None
E=-1.93634995  angles= {(3, 0, 3, 0): 0.04710623158006605, (2, 1, 2, 1): -0.15248204298221218, (3, 1, 3, 1): 0.003034757883097469, (2, 0, 2, 0

#### We can check its consistency using random search

In [ ]:
n=5
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

- QCC

In [6]:
hf_reference = hf_occ(2*h4.n_orbitals, h4.n_electrons)


H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h4.n_orbitals,
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))




Hamiltonian has 105 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1045
2 : 0.0481
3 : 0.0434
4 : 0.0335
5 : 0.0
6 : 0.0


#### We can see that 4 entangulars will be sufficient

In [7]:
n_ents = 4

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h4.n_orbitals)
print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

#Mean-field part of U (Omega):
U_MF = construct_QMF_ansatz(n_qubits = 2*h4.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))


Selected entanglers:
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X6 X7]
1.0 [X2 Y3 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 40 expectationvalues

active variables : 20

E=-1.93303733  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0}  samples= None
E=-1.94498346  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, g

## NH3

- FCI w/ STO-3G

In [12]:

xyz_data = get_molecular_data('nh3', geometry=1.5, xyz_format=True)
basis='sto-3g'

nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*nh3.n_orbitals))

E_FCI = nh3.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 16 

FCI energy: -55.32940782498641


- FCI w/6-31G

In [13]:
basis='6-31g'

nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)


print(list(map(lambda item: item[1][0], nh3.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, nh3.orbitals_by_irrep.items())))

[QuantumChemistryPsi4.OrbitalData(irrep="A'", idx_irrep=0, idx_total=0, energy=-15.682567409874116), QuantumChemistryPsi4.OrbitalData(irrep='A"', idx_irrep=0, idx_total=2, energy=-0.47571264826135573)]
[-15.682567409874116, -0.47571264826135573]


In [14]:
active = {'A\'': [2, 3], 'A"': [0, 1]}
nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*nh3.n_orbitals))

E_FCI = nh3.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -55.91582659486136


- UCCSD w/6-31G


In [18]:
H = nh3.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = nh3.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')



Hamiltonian has 173 terms


Number of UCCSD amplitudes: 5 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 160 expectationvalues

active variables : 5

E=-55.89755128  angles= {(3, 0, 3, 0): 0.0, (3, 2, 3, 1): 0.0, (3, 1, 3, 1): 0.0, (3, 1, 3, 2): 0.0, (3, 2, 3, 2): 0.0}  samples= None
E=-55.91076011  angles= {(3, 0, 3, 0): -0.21731948852539062, (3, 2, 3, 1): -0.030841827392578125, (3, 1, 3, 1): -0.04653167724609375, (3, 1, 3, 2): -0.030841827392578125, (3, 2, 3, 2): -0.061626434326171875}  samples= None
E=-55.91581393  angles= {(3, 0, 3, 0): -0.14209939410142164, (3, 2, 3, 1): -0.024074677804474108, (3, 1, 3, 1): -0.02166660543360647, (3, 1, 3, 2): -0.02407467780447411, (3, 2, 3, 2): -0.04620084889566002}  samples= None
E=-55.91581903  angles= {(3, 0, 3, 0): -0.14

- QCC w/6-31G

In [15]:
hf_reference = hf_occ(2*nh3.n_orbitals, nh3.n_electrons)

H = nh3.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


Hamiltonian has 173 terms



In [16]:
#Define number of entanglers to enter ansatz
n_ents = 5

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*nh3.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*nh3.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

U_MF = construct_QMF_ansatz(n_qubits = 2*nh3.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1087
2 : 0.0308
3 : 0.0233
4 : 0.0154
5 : 0.0154
6 : 0.0
7 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X4 Y5 X6 X7]
1.0 [X2 Y3 X6 X7]
1.0 [X2 Y5 X6 X7]
1.0 [X3 Y4 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 42 expectationvalues

active variables : 21

E=-55.89755128  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 3.141592653589793, gamma_4: 0.0, beta_5: 3.141592653589793, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0}  samples= None
E=-55.90874580  angles= {beta_0: 3.141592653589793, gamma_0: 0.0

## N2

- FCI w/STO-3G

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients
trotter_steps = 1


xyz_data = get_molecular_data('n2', geometry=1.5, xyz_format=True)
basis='sto-3g'

n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*n2.n_orbitals))

E_FCI = n2.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))


Number of spin-orbitals (qubits): 20 

FCI energy: -107.58163492166018


- FCI w/6-31G

In [2]:
basis='6-31g'

n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)
print(list(map(lambda item: item[1][0], n2.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, n2.orbitals_by_irrep.items())))

[QuantumChemistryPsi4.OrbitalData(irrep='AG', idx_irrep=0, idx_total=0, energy=-15.806613506138117), QuantumChemistryPsi4.OrbitalData(irrep='B1U', idx_irrep=0, idx_total=1, energy=-15.805900361433766), QuantumChemistryPsi4.OrbitalData(irrep='B2U', idx_irrep=0, idx_total=5, energy=-0.4968324586317101), QuantumChemistryPsi4.OrbitalData(irrep='B3U', idx_irrep=0, idx_total=6, energy=-0.49683245863119674), QuantumChemistryPsi4.OrbitalData(irrep='B3G', idx_irrep=0, idx_total=7, energy=-0.005793047017178504), QuantumChemistryPsi4.OrbitalData(irrep='B2G', idx_irrep=0, idx_total=8, energy=-0.005793047016557085)]
[-15.806613506138117, -15.805900361433766, -0.4968324586317101, -0.49683245863119674, -0.005793047017178504, -0.005793047016557085]


In [3]:
active = {'AG': [1, 2], 'B1U': [1], 'B2U': [1]}
n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*n2.n_orbitals))

E_FCI = n2.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -108.62442717371962


- UCCSD w/6-31G

In [4]:
H = n2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = n2.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')



Hamiltonian has 105 terms


Number of UCCSD amplitudes: 5 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 160 expectationvalues

active variables : 5

E=-108.62411657  angles= {(3, 0, 3, 0): 0.0, (3, 1, 3, 1): 0.0, (3, 2, 3, 0): 0.0, (3, 0, 3, 2): 0.0, (3, 2, 3, 2): 0.0}  samples= None
E=-108.61512395  angles= {(3, 0, 3, 0): -0.03958892822265625, (3, 1, 3, 1): -0.04364013671875, (3, 2, 3, 0): -0.00821685791015625, (3, 0, 3, 2): -0.00821685791015625, (3, 2, 3, 2): -0.01963043212890625}  samples= None
E=-108.62442311  angles= {(3, 0, 3, 0): -0.0060732169145442565, (3, 1, 3, 1): -0.0066947004723825684, (3, 2, 3, 0): -0.0012605231483839208, (3, 0, 3, 2): -0.0012605231483839208, (3, 2, 3, 2): -0.0030114448103916694}  samples= None
E=-108.62433384  angles= {(3, 0, 3, 0)

- QCC w/6-31G

In [5]:
hf_reference = hf_occ(2*n2.n_orbitals, n2.n_electrons)

H = n2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


Hamiltonian has 105 terms



In [7]:
#Define number of entanglers to enter ansatz
n_ents = 5

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*n2.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0218
2 : 0.0198
3 : 0.0098
4 : 0.0041
5 : 0.0041


In [8]:
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*n2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

U_MF = construct_QMF_ansatz(n_qubits = 2*n2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))


Selected entanglers:
1.0 [X2 Y3 X6 X7]
1.0 [X0 Y1 X6 X7]
1.0 [X4 Y5 X6 X7]
1.0 [X0 Y5 X6 X7]
1.0 [X1 Y4 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 42 expectationvalues

active variables : 21

E=-108.62411657  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 3.141592653589793, gamma_4: 0.0, beta_5: 3.141592653589793, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0}  samples= None
E=-108.62430214  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4